<a href="https://colab.research.google.com/github/Nuri-Tas/Time-Series/blob/main/fb_prophet_forecasting_introduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np 
import pandas as pd 

import matplotlib.pyplot as plt 
import seaborn as sns

%matplotlib inline

Import the cab company profit dataset.

In [160]:
df = pd.read_csv('/content/master.csv', index_col=0, parse_dates=['Date of Travel'], usecols=['Date of Travel', 'Company', 'Profit'])
df.head()

,Company,Profit
Date of Travel,,
2016-01-02,Yellow Cab,137.1860
2016-01-02,Yellow Cab,25.0484
2016-01-02,Yellow Cab,577.9420
2016-01-02,Yellow Cab,233.9280
2016-01-02,Yellow Cab,635.8876


We have two unique companies in the dataset, and we will only forecast profits for the Yellow Cab.

In [161]:
df.Company.unique()

array(['Yellow Cab', 'Pink Cab'], dtype=object)

As we have several rows for the same date, resample the dataframe by taking the mean profit for each day, then unstack the dataframe so that we will have two seperate columns for the two companies' profit.

In [162]:
df = df.groupby([pd.Grouper(freq='D'), 'Company']).Profit.mean().unstack(-1).rename_axis(None, axis=1).reset_index()
df

,Date of Travel,Pink Cab,Yellow Cab
0,2016-01-02,117.302951,323.540294
1,2016-01-03,204.241962,305.154990
2,2016-01-04,107.147750,173.819105
3,2016-01-05,124.226500,276.539405
4,2016-01-06,101.607870,217.016819
...,...,...,...
1090,2018-12-27,65.138333,111.339890
1091,2018-12-28,85.156221,160.401622
1092,2018-12-29,99.487174,156.847740
1093,2018-12-30,71.768857,97.383441


**Forecasting**

Let's forecast profit for Yellow Cab for the last 6 months. Note that `prophet` requires the dataframe to have only date and value columns named `ds` and `y`, respectively.

In [163]:
yellow = df.iloc[:, [0,1]]

# change the column names
yellow.columns = ['ds', 'y']

train = yellow.iloc[:-180]
test = yellow.iloc[-180:]

Fit the model to `yellow` except for the last 180 days to compare the model's predictions with actual values.

In [164]:
from prophet import Prophet

m = Prophet()
m.fit(train)

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmp1ihawbgw/y_phxbai.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp1ihawbgw/b7ycv3dk.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.7/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=59758', 'data', 'file=/tmp/tmp1ihawbgw/y_phxbai.json', 'init=/tmp/tmp1ihawbgw/b7ycv3dk.json', 'output', 'file=/tmp/tmpyc1kbgw0/prophet_model-20220822071816.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
07:18:16 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
07:18:16 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


Extend the train dataframe ds column by 180 days with `make_future_dataframe`

In [165]:
future = m.make_future_dataframe(180)
future

,ds
0,2016-01-02
1,2016-01-03
2,2016-01-04
3,2016-01-05
4,2016-01-06
...,...
1090,2018-12-27
1091,2018-12-28
1092,2018-12-29
1093,2018-12-30


Now, calling `m.predict` upon future will fill future dataframe with predicted values called `yhat` additionally with uncertainty parameters such as `yhat_lower` and `yhat_upper`.

In [166]:
forecast = m.predict(future)
forecast[['ds', 'trend', 'yhat_lower', 'yhat_upper']].tail()

,ds,trend,yhat_lower,yhat_upper
1090,2018-12-27,36.634472,51.789159,93.727220
1091,2018-12-28,36.595640,67.951363,109.529584
1092,2018-12-29,36.556808,65.174836,108.009721
1093,2018-12-30,36.517976,75.833686,117.665957
1094,2018-12-31,36.479144,48.352939,88.301006


You can get all the predicted values or the last week, month, or yearly values with the following interactive figure:

In [167]:
%matplotlib inline
from prophet.plot import plot_plotly, plot_components_plotly

plot_plotly(m, forecast)

We can additionally have the trend, and yearly and weekly components of results:

In [168]:
plot_components_plotly(m, forecast)

Component plots showed that there is a steady downward trend in the last few months, and the profits are actually lower during weekdays compared to weekends. 

**Evaluation**

Find the MAE, RMSE, and MAPE values for the predictions:

In [169]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error 

def evaluate(test, forecast, test_size):
    metrics = [mean_absolute_error, mean_squared_error, mean_absolute_percentage_error]
    results = []
    for metric in metrics:
         results.append(metric(test.y, forecast['yhat'].iloc[-test_size:]))
    return results        

In [170]:
results = evaluate(test, forecast, test_size=180)
'MAE:{}, RMSE:{}, MAPE:{}'.format(results[0], np.sqrt(results[1]), results[2]) 

'MAE:11.200069542186288, RMSE:14.186996338671122, MAPE:0.27626895230213855'

RMSE tells us that the predictions are generally within half standard deviation of the actual values:

In [59]:
yellow.y.describe().T

count    1095.000000
mean       53.572769
std        26.249742
min        -2.490146
25%        34.391118
50%        51.085816
75%        69.515052
max       204.241962
Name: y, dtype: float64

**ATM Cash Demanding Forecasting**

Now, we will forecast cash withdrawn forecasting for the following dataset:

In [189]:
atm = pd.read_csv('/content/atm_data_m2.csv')
atm.head()

,Unnamed: 0,atm_name,weekday,festival_religion,working_day,holiday_sequence,trans_date_set,trans_month,trans_year,prevweek_mean,total_amount_withdrawn
0,11,Mount Road ATM,MONDAY,NH,W,WWW,1,1,2011,648600.0,897100
1,16,Mount Road ATM,TUESDAY,NH,W,WWW,1,1,2011,648600.0,826000
2,21,Mount Road ATM,WEDNESDAY,NH,W,WWW,1,1,2011,648600.0,754400
3,26,Mount Road ATM,THURSDAY,NH,W,WWW,2,1,2011,648600.0,834200
4,31,Mount Road ATM,FRIDAY,NH,W,WWW,2,1,2011,648600.0,575300


Add a new column in the 'Year-Month-Day' format and drop unnecessary columns:




In [172]:
atm['date'] = pd.to_datetime('2011-01-03')
for i in range(len(atm)):
  atm['date'][i] = pd.to_datetime('2011-01-03') + pd.Timedelta(days=i)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [173]:
atm = atm[['date', 'total_amount_withdrawn']]
atm.columns = ['ds', 'y']
atm

,ds,y
0,2011-01-03,897100
1,2011-01-04,826000
2,2011-01-05,754400
3,2011-01-06,834200
4,2011-01-07,575300
...,...,...
2239,2017-02-19,447400
2240,2017-02-20,153800
2241,2017-02-21,167100
2242,2017-02-22,317400


Build the model:

In [174]:
def prophet_model(df, test_size):
    train = df.iloc[:-test_size]

    m = Prophet()
    m.fit(train)

    future = m.make_future_dataframe(test_size)
    forecast = m.predict(future)
    return m, forecast

m, forecast = prophet_model(atm, test_size=365)    

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmp1ihawbgw/leuxrcd9.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp1ihawbgw/605fynjb.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.7/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=49351', 'data', 'file=/tmp/tmp1ihawbgw/leuxrcd9.json', 'init=/tmp/tmp1ihawbgw/605fynjb.json', 'output', 'file=/tmp/tmpd64gwd4g/prophet_model-20220822071852.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
07:18:52 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
07:18:53 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


In [175]:
plot_plotly(m, forecast)

In [176]:
plot_components_plotly(m, forecast)

In [183]:
test_size = 365
test = atm.iloc[-test_size:]
results = evaluate(test, forecast, test_size)
'MAE:{}, RMSE:{}, MAPE:{}'.format(results[0], np.sqrt(results[1]), results[2]) 

'MAE:231636.92403052098, RMSE:288103.8141353373, MAPE:10.721614368878043'

Note that although the evaluation metrics aren't great, this can be partly attributed to the fact that train and test set showed very different characteristics, and the dataset has gone under a rational downward trend for the last few months.

In [184]:
test.y.describe()

count    3.650000e+02
mean     2.637556e+05
std      1.901031e+05
min      1.000000e+02
25%      1.272000e+05
50%      2.220000e+05
75%      3.610000e+05
max      1.213600e+06
Name: y, dtype: float64

In [188]:
atm.iloc[:-test_size].y.describe()

count    1.879000e+03
mean     5.646616e+05
std      2.367109e+05
min      4.000000e+02
25%      4.184000e+05
50%      5.562000e+05
75%      7.152000e+05
max      1.410700e+06
Name: y, dtype: float64